In [1]:
%matplotlib notebook
# starter code
from tqdm import tqdm
import networkx as nx
from graph_tool.all import load_graph, shortest_distance
from ic import simulate_cascade, observe_cascade, get_gvs, get_o2src_time, get_infection_time, gen_nontrivial_cascade
from utils import get_rank_index

gtype = 'balanced-tree'
g = load_graph('data/{}/2-6/graph.gt'.format(gtype))
gnx = nx.read_graphml('data/{}/2-6/graph.graphml'.format(gtype))
gnx = nx.relabel_nodes(gnx, {i: int(i) for i in gnx.nodes_iter()})
N1, N2 = 100, 100
p, q = 0.5, 0.2

gvs = get_gvs(g, p, N2)

In [2]:
from mwu import mwu
infection_times, source, obs_nodes = gen_nontrivial_cascade(
    g, p, q)
print(len(obs_nodes))
r, q_log, mu_log, is_nbr_log = mwu(g, gvs,
        source, obs_nodes, infection_times, o2src_time=None,
        active_method='max_mu',
        reward_method='dist',
        eps=0.2,
        use_uninfected=True,
        debug=True, save_log=True)

 33%|███▎      | 1/3 [00:00<00:00,  8.77it/s]

3


100%|██████████| 3/3 [00:00<00:00,  8.69it/s]
0it [00:00, ?it/s]/home/cloud-user/code/source_finding/rewards.py:22: RuntimeWarning: invalid value encountered in true_divide
  / counts)
3it [00:00, 987.44it/s]


t1=2, t2=1
source reward: 1.00
t1=2, t2=6
source reward: 1.00
t1=1, t2=6
source reward: 1.00
query 21
using pairs to update sll
source reward (without smoothing): 1.00
max reward: 1.0
probas [ 0.          0.5         0.          0.33333333  0.66666667  0.          0.
  0.16666667  0.          0.75      ]
new sll[source] = 0.18745337535802775
source reward (without smoothing): 1.00
max reward: 1.0
probas [ 0.85714286  0.8         0.88888889  0.85714286  0.66666667  0.
  0.90909091  0.88888889  0.88888889  0.8       ]
new sll[source] = 0.21931478050199774
source reward (without smoothing): 1.00
max reward: 1.0
probas [ 0.66666667  0.5         0.75        0.66666667  0.          0.          0.8
  0.75        0.75        0.5       ]
new sll[source] = 0.2736860289949278
add q to ref_nodes (#nodes=4)
source current rank = 61, 0.00
using node from nodes_to_use
46 is uninfected
source current rank = 61, 0.00
using node from nodes_to_use
19 is uninfected
source current rank = 61, 0.00
using nod

/home/cloud-user/code/source_finding/rewards.py:29: RuntimeWarning: invalid value encountered in true_divide
  / (np.absolute(len1 + len2)))


source current rank = 61, 0.00
query 40
40 is uninfected
source current rank = 61, 0.00
query 22
22 is uninfected
source current rank = 61, 0.00
using node from nodes_to_use
using pairs to update sll
source reward (without smoothing): 1.00
max reward: 1.0
probas [ 0.83333333  0.75        0.          0.5         1.          0.          0.
  0.25        0.66666667  1.        ]
new sll[source] = 0.0
source reward (without smoothing): 1.00
max reward: 1.0
probas [ 0.8         0.66666667  0.85714286  0.8         0.          0.88888889
  0.88888889  0.85714286  0.85714286  0.66666667]
new sll[source] = 0.0
source reward (without smoothing): 1.00
max reward: 1.0
probas [ 0.66666667  0.5         0.75        0.66666667  0.          0.          0.8
  0.75        0.75        0.5       ]
new sll[source] = 0.0
add q to ref_nodes (#nodes=4)
source current rank = 61, 0.00
using node from nodes_to_use
20 is uninfected
source current rank = 61, 0.00
query 1
using pairs to update sll
source reward (with

In [3]:
infection_times[q_log]

PropertyArray([ 0,  1, -1, -1, -1, -1, -1, -1, -1,  2,  3, -1, -1, -1,  4,  4, -1,
       -1,  5,  1, -1, -1,  5, -1, -1, -1,  6, -1, -1, -1, -1, -1, -1,  6,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], dtype=int32)